<a href="https://colab.research.google.com/github/OswaldoLopezAlcaraz/TI_1_Practica3/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Oswaldo López Alcaraz

**e-mail:** oswaldo.lopez5103@alumnos.udg.mx

## MODULES


In [ ]:
import numpy as np
import pandas as pd
import math
from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

import plotly.graph_objects as go
import plotly.figure_factory as ff

from scipy.spatial import distance

## CLASSES


In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [50]:
from sqlalchemy import false

###################### Function to create Brownian walks #######################
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Summary:
    This function generates a Brownian Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk.
    speed: number of units per steps.
    s_pos: coordinates (x and y ) where the walk starts.
  Returns:
    BM_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df


################## Function to create Correlated Random Walks ##################
def crw_2d(n_steps=1000, speed=6, s_pos=[0,0], crw_exp=0.3):
  """
  Summary:
    This function generates a Correlated Random Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk
    speed: number of units per steps
    s_pos: coordinates (x and y ) where the walk starts.
    crw_exp:  exponent used in the Cauchy distribution
  Returns:
    CRW_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init of velocity vector
  velocity = Vec2d(speed,0)

  CRW_2d_df = pd.DataFrame( columns= ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos': s_pos[1]},])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  for i in range(1, n_steps):

    # Determination of angle using cauchy distribution
    turn_angle = wrapcauchy.rvs(crw_exp)
    velocity = velocity.rotated(turn_angle)

    # Motion of Brownian walker
    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i-1]+velocity.x,
                           'y_pos':CRW_2d_df.y_pos[i-1]+velocity.y,}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)

  return CRW_2d_df


########## Function to obtain Euclidian distance between two points ############
def my_distance_function(origin, destination):
  """
  Arguments:
    origin: x & y coordinates that represent the origin of a trajectory
    destination: x & y coordinate representing the destination of a trajectory
  Returns:
    Euclidiean distance between two points in the form of int or float.
  """
  return math.sqrt(((destination.x_pos - origin.x_pos)**2) +
                    (( destination.y_pos - origin.y_pos)**2) )

################# Function to get Squared Mean Displacement ####################
def smd(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe of length motion.df.shape[0] - 1 containing SMD in for each step
  """
  smd = pd.DataFrame(columns=['MSD'])

  for n in range(1,motion_df.shape[0]):

    displacement_vec = np.array([(my_distance_function(motion_df.iloc[i-n], motion_df.iloc[i]))**2
                              for i in range(n, motion_df.shape[0],1)])
    tmp_df = pd.DataFrame([{'MSD':np.mean(displacement_vec)}])
    smd = pd.concat([smd,tmp_df], ignore_index=True)

  return smd


##################### Function to create Brownian walks ########################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array([pos_b.x_pos - pos_a.x_pos , pos_b.y_pos - pos_a.y_pos ])
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array([pos_c.x_pos - pos_b.x_pos , pos_c.y_pos - pos_b.y_pos ])
    norm_bc = np.linalg.norm(vec_bc)

    dot_p =np.dot(vec_ab , vec_bc)

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

########################Function to automate the obten
def turning_angles_producer(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe with turning angles for each step.
  """
  trn_angle_df = pd.DataFrame(columns = ['TA_CRW'])

  for i in range(motion_df.shape[0]-2):
    ta = turning_angle(motion_df.iloc[i], motion_df.iloc[i+1], motion_df.iloc[i+2])
    tmp_df = pd.DataFrame([{'TA_CRW': ta }])
    trn_angle_df = pd.concat([trn_angle_df, tmp_df], ignore_index=True)

  return trn_angle_df



 ############# Function to get the Step Length metrics of any walk #############
def step_length_determiner(motion_df):

  ## Init of variables
  step_lengths_df = pd.DataFrame(columns = ['count','Turn_angle','Step_length'])
  is_TA_zero = False
  steps_total = motion_df.shape[0] -3
  passed_ta = 0
  overall_steps = 0
  big_index = 0
  temp_steps_accum = 0
  count_index = 0
  # Try that will catch the IndexError when algorithm goes out of index limits.
  try:
    # First iteration through all the points.
    while big_index <= steps_total:
      # First angle check.
      t_a = turning_angle(motion_df.iloc[big_index],
                            motion_df.iloc[big_index +1],
                            motion_df.iloc[big_index + 2])
      is_TA_zero = t_a == 0.00
      # Verifying that a step needs to be added and "virtual counts" resets
      if not is_TA_zero:
        if passed_ta == 0:  #When the last iteration ta is 0 and this iteration ta != 0
                            # a new step is added and "virtual counts" resets
          temp_steps_accum += 1
          tmp_df = pd.DataFrame([{'count': count_index ,
                              'Turn_angle':t_a,
                              'Step_length': temp_steps_accum}])
          step_lengths_df = pd.concat([step_lengths_df,tmp_df],ignore_index=True)
          temp_steps_accum = 0
          count_index += 1
          passed_ta = t_a
          big_index +=1
          overall_steps += 1

        else:  # This block of code should be eliminated since both results do the same.
              # the if-else should be taken off and just leave the content of it
          temp_steps_accum += 1

          tmp_df = pd.DataFrame([{'count': count_index ,
                            'Turn_angle':t_a,
                            'Step_length': temp_steps_accum}])
          step_lengths_df = pd.concat([step_lengths_df,tmp_df],ignore_index=True)

          temp_steps_accum = 0
          count_index += 1
          passed_ta = t_a
          big_index +=1
          overall_steps += 1

      else:  ## Following is the code used when there an zero angle is present.
        temp_steps_accum += 1
        overall_steps += 1
        while is_TA_zero:  # Second iteration corresponding to streight line walk

          t_a = turning_angle(motion_df.iloc[big_index + temp_steps_accum],
                              motion_df.iloc[big_index + temp_steps_accum + 1],
                              motion_df.iloc[big_index + temp_steps_accum + 2])
          is_TA_zero = t_a == 0.00

          ## block of code that verify the condition of second iterations continues
          ## steps are added virtualy in the second iteration.
          if is_TA_zero:
            temp_steps_accum += 1
            overall_steps += 1
          else:  ## If an angle bigger than zero comes, big index is adjust for the
                 ## first iteration
            big_index += temp_steps_accum
            passed_ta = t_a
  except IndexError:
    pass

  return step_lengths_df



## BROWNIAN WALK

In [ ]:
# Creation of walk with help of bm_2d() function.
BM_2d_df = bm_2d(2000, 3, [5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_2d_df.x_pos,
    y = BM_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='BM_2d_trajectory',
    showlegend=True
))

fig_BM_2d.show()

## CORRELATED RANDOM WALK

In [ ]:
# Creation of CRW through use of crw_2d() function.
CRW_2d_df= crw_2d(100, 3 , [0,0], 0.3)

# 2D plotting
fig_BM_pdf = go.Figure()

fig_BM_pdf.add_trace(go.Scatter(x = CRW_2d_df.x_pos,
                                y = CRW_2d_df.y_pos,
                                marker = dict(size=2),
                                line = dict(width=1),
                                mode = 'lines',
                                name = 'CB_Walker',
                                showlegend = True))

fig_BM_pdf.show()

## METRICS

## Actividad 1: Path-Length - (BM1 vs BM2 vs CRW)

In [ ]:
# Obtention of trajectories
BM_2d_df_1 = bm_2d(1000, 1, [0,0])
BM_2d_df_2 = bm_2d(1000, 2, [0,0])
CRW_2d_df_n = crw_2d(1000, 3, [0,0], 0.3)

#CRW_2d_df_6_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Production of path-length metrics through user-made function: my_distance_function()
dis_BM_1 = np.array([my_distance_function(BM_2d_df_1.iloc[i-1],BM_2d_df_1.iloc[i]) for i in range(1,BM_2d_df_1.shape[0])])
pl_BM_1 = np.cumsum(dis_BM_1)

dis_BM_2 = np.array([my_distance_function(BM_2d_df_2.iloc[i-1],BM_2d_df_2.iloc[i]) for i in range(1,BM_2d_df_2.shape[0])])
pl_BM_2 = np.cumsum(dis_BM_2)

dis_CRW_n = np.array([my_distance_function(CRW_2d_df_n.iloc[i-1],CRW_2d_df_n.iloc[i]) for i in range(1,CRW_2d_df_n.shape[0])])
pl_CRW_n = np.cumsum(dis_CRW_n)


In [ ]:
# Init figure
fig_path_length = go.Figure()

# Plot trajectory
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_1))+1,
    y = pl_BM_1,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_BM_1',
    showlegend=True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_2))+1,
    y = pl_BM_2,
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    name='path_length_BM_2',
    showlegend=True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_n))+1,
    y = pl_CRW_n,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_CRW',
    showlegend=True
))

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (BM vs CRW)

In [ ]:
# Import of trajectories used as reference.
bm_2d_6 = pd.read_csv('trajectories/brownian_6.csv')
crw_2d_6 = pd.read_csv('trajectories/crw_6_9.csv')
# Use of user-made functions for Mean Squared Displacement metrics.
bm_smd = smd(bm_2d_6)
crw_smd = smd(crw_2d_6)

MSD    105547.535505
dtype: float64

In [ ]:
# Graph creation

fig_path_smd = go.Figure()

# Plot smd
fig_path_smd.add_trace(go.Scatter(
    x = np.arange(0,bm_2d_6.shape[0]),
    y = bm_smd.MSD,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='bm_smd',
    showlegend=True
))

fig_path_smd.add_trace(go.Scatter(
    x = np.arange(0,crw_2d_6.shape[0]),
    y = crw_smd.MSD,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='crw_smd',
    showlegend=True
))
fig_path_smd.show()

## Actividad 3: Turning Angle Distribution - (Dist origen vs Dist observada)

---



In [112]:
# Import of trajectories used as reference
crw_2d_6_6 = pd.read_csv('trajectories/crw_6_6.csv')
crw_2d_6_9 = pd.read_csv('trajectories/crw_6_9.csv')

# Use of user-made function to get turning angle metrics
trn_angle_df_6_6 = turning_angles_producer(crw_2d_6_6)
trn_angle_df_6_9 = turning_angles_producer(crw_2d_6_9)

resolution = 100
aux_domain = np.linspace(-3, 3, resolution)
crw_exp_6 = 0.6
crw_exp_9 = 0.9

# Production of distributions
cauchy_dist_6 = np.array([wrapcauchy.pdf(abs(i), crw_exp_6)for i in aux_domain])
cauchy_dist_9 = np.array([wrapcauchy.pdf(abs(i), crw_exp_9)for i in aux_domain])


In [113]:
# Graph creation
fig = go.Figure()
fig.add_trace(go.Histogram(x=trn_angle_df_6_6['TA_CRW'],
                           opacity=0.8,
                           name ='Observed_0.6',
                           histnorm='probability density',
                           marker_color='#434cf7',
                           nbinsx=130))
fig.add_trace(go.Histogram(x=trn_angle_df_6_9['TA_CRW'],
                           opacity=0.8,
                           name='Observed_0.9',
                           histnorm='probability density',
                           nbinsx=130,
                           marker_color='#eb9489'
                           ))

fig.add_trace(go.Scatter( x = aux_domain,
                          y = cauchy_dist_6,
                          marker = dict(size=2),
                          line = dict(width=2),
                          mode='lines',
                          name='Cauchy_6_6',
                          marker_color='#0b13a1',
                          showlegend=True))

fig.add_trace(go.Scatter( x = aux_domain,
                          y = cauchy_dist_9,
                          marker = dict(size=2),
                          line = dict(width=2),
                          mode='lines',
                          name='Cauchy_6_9',
                          marker_color='#d92811',
                          showlegend=True))

# The two histograms are drawn on top of another
fig.update_layout(barmode='overlay')
fig.show()

## Actividad 4: Step-length Distribution - (Dist origen vs Dist observada)

In [ ]:
# Import of trajectories and metrics used as reference.
lf_2d_6_1 = pd.read_csv('trajectories/levy_6_1.csv')
lf_2d_6_7 = pd.read_csv('trajectories/levy_6_7.csv')

#met_df_5 = pd.read_csv('metrics/met_df_5.csv')
#met_df_4 = pd.read_csv('metrics/met_df_4.csv')

# Use of made-by-user function to get the Step_length
levy_ls_2d_6_1 = step_length_determiner(lf_2d_6_1)
levy_ls_2d_6_7 = step_length_determiner(lf_2d_6_7)


In [137]:
# Graphs creation

resolution = 100
aux_domain = np.linspace(0, 1200, resolution)

# pdf (i,alpha,beta,miu)
levy_dist_6_1 = np.array([levy_stable.pdf(i,alpha=1.0,beta=1.0,loc=6) for i in aux_domain])
levy_dist_6_7 = np.array([levy_stable.pdf(i,alpha=0.7,beta=1.0,loc=6) for i in aux_domain])

layout = go.Layout(yaxis=dict(range=[0,.010]),
                    xaxis=dict(range=[-100, 1200]))

fig = go.Figure(layout=layout)
fig.add_trace(go.Histogram(x=levy_ls_2d_6_1['Step_length'],
                           opacity=0.8,
                           name ='Observed_alpha=1.0_beta=1.0',
                           histnorm='probability density',
                           marker_color='#434cf7',
                           ))

fig.add_trace(go.Histogram(x=levy_ls_2d_6_7['Step_length'],
                           opacity=0.8,
                           name='Observed_alpha=0.7_beta=1.0',
                           histnorm='probability density',
                           marker_color='#eb9489'))

fig.add_trace(go.Scatter( x = aux_domain,
                          y = levy_dist_6_1,
                          marker = dict(size=2),
                          line = dict(width=2),
                          mode='lines',
                          name='Levy_alpha=1.0',
                          marker_color='#0b13a1',
                          showlegend=True
                          ))

fig.add_trace(go.Scatter( x = aux_domain,
                          y = levy_dist_6_7,
                          marker = dict(size=2),
                          line = dict(width=2),
                          mode='lines',
                          name='Levy_alpha=0.7',
                          marker_color='#d92811',
                          showlegend=True))

fig.show()